# 使用`.pyx`改写python程序

在纯净模式中,我的代码实现手段很有限:

1. 只能直接使用python模块和对象
2. 无法使用c++的容器和算法
3. 无法使用cimport

In [1]:
%load_ext Cython

In [2]:
%%cython
# distutils: language=c++

from libcpp.vector cimport vector
from libcpp.string cimport string
#from libc.stdio cimport printf

cpdef print_vect(string content):
    cdef vector[int] vect
    cdef int i
    for i in range(10):
        vect.push_back(i)
    for i in vect:
        #printf("%d",vect[i])
        print(vect[i])
    for i in content:
        #printf("%d",vect[i])
        print(i)

In [3]:
print_vect("黑龙江".encode())

0
1
2
3
4
5
6
7
8
9
-23
-69
-111
-23
-66
-103
-26
-79
-97


# Cython语法

Cython是python的超集,所以python解释器无法解释cython代码,必须编译才可以.


## 静态化参数

Cython是一个Python编译器。这意味着它可以编译普通的Python代码，而不会有任何改变（除了一些尚未支持的语言功能之外，还有一些明显的例外）.然而，对于性能关键代码，添加静态类型声明通常是有帮助的，因为它们将允许Cython退出Python代码的动态特性，并生成更简单更快的C代码.但是，必须注意的是，类型声明会使源代码更加冗长，因而可读性更低.因此，不要在没有正当理由的情况下使用它们，例如基准测试证明他们在性能关键部分真正使代码更快速地使用它们是不鼓励的.通常在正确的地方有几种类型可以走很长的路.所有C类型都可用于类型声明:整数和浮点类型，复数，结构体，联合和指针类型. Cython可以在分配类型之间自动正确地进行转换.这也包括Python的任意大小的整数类型，其中转换为C类型的值溢出会在运行时引发Python `OverflowError`.(但是，在进行算术时，不会检查是否溢出).在这种情况下，生成的C代码将正确安全地处理C类型的依赖于平台的大小.

### 在python函数中使用c语言的类型指明翻译为C语言后的参数类型

由于这些参数被传递到Python声明的函数中，它们会转换为指定的C类型值.但这只适用于数字和字符串类型

In [4]:
%%cython

def f(double x):
    return x**2-x

def integrate_f(double a, double b, int N):
    cdef int i
    cdef double s, dx
    s = 0
    dx = (b-a)/N
    for i in range(N):
        s += f(a+i*dx)
    return s * dx

In [5]:
f(1.2)

0.24

## C级别申明

作为一种动态语言，Python鼓励了一种根据方法和属性考虑类和对象的编程风格，而不仅仅局限于类层次结构中.这可以使Python成为一种非常轻松和舒适的语言，用于快速开发，但有一个代价 -- 管理数据类型的"繁文缛节"被转储到翻译器上.在运行时，解释器在搜索命名空间，获取属性和解析参数和关键字元组方面做了大量工作。与"早期绑定"语言（如C++）相比，这种运行时"后期绑定"是Python相对较慢的主要原因.然而使用Cython可以通过使用"早期绑定"编程技术获得显着的加速.

### `cdef`语句用于创建C级声明

+ 申明变量

```cython
cdef int i, j, k
cdef float f, g[42], *h
```

+ 申明结构体

```cython
cdef struct Grail:
    int age
    float volume
```

+ 申明联合体

```cython
cdef union Food:
    char *spam
    float *eggs
```

+ 申明枚举

```cython
cdef enum CheeseType:
    cheddar, edam,
    camembert
```

+ 申明函数

```cython
cdef int eggs(unsigned long l, float f):
```

`cdef`关键字指定早期绑定,默认是私有的,只有在申明时指定public才会暴露

## 类型转换

在cython中使用`<xxx>yyy`操作符来进行类型转换,其使用方式与C中类似.

```cython
cdef char *p, float *q
p = <char*>q

```

值得注意的是cython中python的`bool`类型会转化为`bint`,而python中的自定义类的实例则对应的`object`

### 类型检测

和C中类似,类型转换时使用`<xxx?>yyy`会先进行检测

## 字符串

C级别无论是字符数组还是c++的string,都只接收python的bytes作为转换来源.返回的也只会转换为bytes.因此需要注意.

## 函数,方法申明

Cython中有三种类型的函数声明.

### Python的可调用对象(def)

这种类型的函数特点:

+ 使用`def`申明
+ 可以被Python解释器调用
+ 可以被Python对象调用
+ 返回Python对象
+ 参数可以静态化

###  C的可调用对象 (cdef)

这种类型的函数特点:

+ 用`cdef`语句声明
+ 无法在python解释器中访问
+ 可以被Python对象或C值调用
+ 内部变量必须申明
+ 可以返回Python对象或C值

### Python和C的可调用(cpdef)

这种类型的函数特点:

+ 用cpdef语句声明.
+ 可以从任何地方调用
+ 当从其他Cython代码调用时,使用更快的C调用约定


### cython的内置函数


Cython将对大多数内置函数的调用编译为对相应的Python / C API例程的直接调用,使它们特别快。
只有使用这些名称的直接函数调用已优化.如果你使用这些名称中的一个假设它是一个Python对象，例如将它分配给一个Python变量，然后调用它，那么该调用将作为一个Python函数调用.

内置函数|返回类型|相当于Python/C API中的类型
---|---|---
`abs(obj)`|	object, double, ...|PyNumber_Absolute, fabs, fabsf, ...
`callable(obj)`|bint|PyObject_Callable
`delattr(obj, name)`|None|PyObject_DelAttr
`exec(code, [glob, [loc]])`|object	
`dir(obj)`|	list|PyObject_Dir
`divmod(a, b)`|	tuple|PyNumber_Divmod
`getattr(obj, name, [default])`|object|PyObject_GetAttr
`hasattr(obj, name)`|bint|PyObject_HasAttr
`hash(obj)`|int / long|PyObject_Hash
`intern(obj)`|object|`Py*_InternFromString`
`isinstance(obj, type)`|bint|PyObject_IsInstance
`issubclass(obj, type)`|bint|PyObject_IsSubclass
`iter(obj, [sentinel])`|object|PyObject_GetIter
`len(obj)`|	Py_ssize_t|PyObject_Length
`pow(x, y, [z])`|object|PyNumber_Power
`reload(obj)`|object|PyImport_ReloadModule
`repr(obj)`|object|PyObject_Repr
`setattr(obj, name)`|void|PyObject_SetAttr

## 类申明(扩展类型)

cython扩展类型可以使用`cdef class`来定义.

### 属性

其中的元素可以使用`cdef`来定义,默认是私有的,但可以使用`public`或者`readonly`关键字指定其封装形式.

In [6]:
%%cython
cdef class Rectangle:
    cdef public int x0
    cdef readonly int y0
    cdef int x1, y1
    def __init__(self, int x0, int y0, int x1, int y1):
        self.x0 = x0; self.y0 = y0; self.x1 = x1; self.y1 = y1
        
    cdef int _area(self):
        cdef int area
        area = (self.x1 - self.x0) * (self.y1 - self.y0)
        if area < 0:
            area = -area
        return area

    def area(self):
        return self._area()
       
def rectArea(x0, y0, x1, y1):
    rect = Rectangle(x0, y0, x1, y1)
    return rect.area()

In [7]:
r = Rectangle(1, 2, 3, 1)

In [8]:
r.x0

1

In [9]:
r.y0

2

In [10]:
r.x1

AttributeError: '_cython_magic_31443cea76eb5b8780ecd933e92cc406.Rec' object has no attribute 'x1'

In [11]:
r.x0=2
r.x0

2

In [12]:
r.y0 = 4

AttributeError: attribute 'y0' of '_cython_magic_31443cea76eb5b8780ecd933e92cc406.Rectangle' objects is not writable

In [13]:
r.area()

1

In [14]:
r._area()

AttributeError: '_cython_magic_31443cea76eb5b8780ecd933e92cc406.Rec' object has no attribute '_area'

### 特性

cython特性除了一般python语句一样的装饰器语法外,还可以使用如下特殊定义方式,两者效果一致.

```cython
cdef class Spam:

    property cheese:

        "A doc string can go here."

        def __get__(self):
            # This is called when the property is read.
            ...

        def __set__(self, value):
            # This is called when the property is written.
            ...

        def __del__(self):
            # This is called when the property is deleted.
```

`__get__()`，`__set__()`和`__del__()`方法都是可选的.如果省略，属性访问会引发异常.


不推荐这种写法,因为看起来和python差别太大

In [15]:
%%cython
cdef class CheeseShop:

    cdef object cheeses

    def __cinit__(self):
        self.cheeses = []

    property cheese:   # note that this syntax is deprecated

        def __get__(self):
            return "We don't have: %s" % self.cheeses

        def __set__(self, value):
            self.cheeses.append(value)

        def __del__(self):
            del self.cheeses[:]

In [16]:
shop = CheeseShop()
print(shop.cheese)

shop.cheese = "camembert"
print(shop.cheese)

shop.cheese = "cheddar"
print(shop.cheese)

del shop.cheese
print(shop.cheese)

We don't have: []
We don't have: ['camembert']
We don't have: ['camembert', 'cheddar']
We don't have: []


### 方法

`Rectangle`中`_area`是C级别的函数,不可被访问,所以需要使用`area`方法来封装.不过通常是使用`cpdef`直接实现的

In [17]:
%%cython
cdef class Rectangle:
    cdef int x0, y0
    cdef int x1, y1
    def __init__(self, int x0, int y0, int x1, int y1):
        self.x0 = x0; self.y0 = y0; self.x1 = x1; self.y1 = y1
        
    cpdef int area(self):
        cdef int area
        area = (self.x1 - self.x0) * (self.y1 - self.y0)
        if area < 0:
            area = -area
        return area

def rectArea(x0, y0, x1, y1):
    cdef Rectangle rect
    rect = Rectangle(x0, y0, x1, y1)
    return rect.area()

In [18]:
r = Rectangle(1, 2, 3, 1)

### 方法重载

在扩展类型中同一申明方式的可以相互重载,而不同申明方式的则有一套优先级:
+ `cpdef`可以重载`cdef`,而反过来就不行
+ `def`可以重载`cpdef`,而反过来就不行

In [19]:
%%cython
cdef class A:
    cdef foo(self):
        print("A")
cdef class AA(A):
    cdef foo(self):
        print("AA")
    cpdef bar(self):
        self.foo()
        
        

In [20]:
AA().bar()

AA


In [21]:
%%cython
cdef class A:
    cdef foo(self):
        print("A")

cdef class B(A):
    cpdef foo(self, x=None):
        print("B", x)

class C(B):
    def foo(self, x=True, int k=3):
        print("C", x, k)

In [22]:
B(12).foo()

B None


In [23]:
C().foo()

C True 3


## 继承

基类的完整定义必须可用于Cython，因此如果基类是内置类型，它必须先前已声明为extern扩展类型.如果基类型在另一个Cython模块中定义，则必须声明为extern扩展类型或使用cimport语句导入.

一个扩展类型只能有一个基类(cython的扩展类不支持多重继承).但可以被python类继承,这种继承支持多继承.

有一种方法可以防止扩展类型在Python中被子类化.这是通过`final`指令完成的，通常使用装饰器在扩展类型上设置

```cython
cimport cython
 
@cython.final
cdef class Parrot:
   def done(self): pass
```

## 扩展类型快速实例化

Cython提供了两种方法来加速扩展类型的实例化.

+ 第一个是直接调用`__new__()`特殊静态方法，如从Python中所知。对于例子扩展类型Penguin，可以使用以下代码：

In [24]:
%%cython
cdef class Penguin:
    cdef public object food
 
    def __cinit__(self, food):
        self.food = food
 
    def __init__(self, food):
        print("eating!")
        

In [25]:
normal_penguin = Penguin('fish')

eating!


In [26]:
normal_penguin.food

'fish'

In [27]:
fast_penguin = Penguin.__new__(Penguin, 'wheat') 

In [28]:
fast_penguin.food

'wheat'

`__new__()`实例化的对象会不运行`__init__()`只会运行`__cinit__()`

+ 第二个性能改进适用于经常在一行中创建和删除的类型，以便他们可以从freelist中受益.

    Cython为此提供了装饰器`@cython.freelist(N)`，它为给定类型创建了一个静态大小的N个实例的`freelist`.例：

In [29]:
%%cython
cimport cython
 
@cython.freelist(8)
cdef class Penguin:
    cdef object food
    def __cinit__(self, food):
        self.food = food

In [30]:
penguin = Penguin('fish 1')

In [31]:
penguin = None
penguin = Penguin('fish 2')

## 特殊方法

cython也支持特殊方法,它支持的特殊方法可在[这里看到](http://docs.cython.org/en/latest/src/reference/special_methods_table.html#special-methods-table)


扩展类型的特殊方法必须用`def`，而不是`cdef`声明,这不会影响他们的性能 -- Python使用不同的调用约定来调用这些特殊方法.

这边列举几个比较中要的:

### 初始化方法：`__cinit __()` 和`__init __()`

有两种方法与初始化对象有关.

+ `__cinit__()`

    方法是应该执行对象的基本C级初始化，包括对象将拥有的任何C数据结构的分配.你需要小心你在`__cinit__()`方法中做什么，因为对象可能还不是完全有效的Python对象.因此，你应该小心调用任何Python可能触摸对象的操作,特别是其方法.

    在调用`__cinit__()`方法时，已经为对象分配了内存，并且任何C属性已初始化为0或null。(任何Python属性也被初始化为None，但是你可能不应该依赖它.)你的`__cinit__()`方法一定只会被调用一次.
    
    如果扩展类型有基类，那么在调用`__cinit __()`方法之前，将自动调用基类的`__cinit__()`方法,你不能显式调用基类的`__cinit__()`方法。如果需要将修改的参数列表传递给基类,则必须在`__init__()`方法中执行初始化的相关部分(其中调用继承方法的正常规则适用).

+ `__init__()`

    在`__cinit__()`方法中不能安全完成的任何初始化都应该在`__init__()`方法中完成.当调用`__init __()`时，对象是一个完全有效的Python对象，所有操作都是安全的.在某些情况下,`__init__()`可能被多次调用或根本不被调用.

传递给构造函数的任何参数都将传递给`__cinit __()`方法和`__init__()`方法。如果你预计在Python中继承扩展类型,可能会将参数以`*和**参数`的形式传给`__cinit__()`方法，以便它可以接受和忽略额外的参数.否则，任何具有带有不同签名的`__init __()`的Python子类都必须覆盖`__new __()`以及`__init __()`，明显这样很不友好.或者，为了方便起见，如果你声明你的`__cinit__()`方法没有参数(除了`self`)，它将简单地忽略传递给构造函数的任何额外的参数.这种方式可能更加保险.

## 析构方法：`__dealloc __()`

与`__cinit__()`方法的逆向方法是`__dealloc__()`方法。`__cinit__()`方法中显式分配内存的任何C数据(例如通过malloc分配的空间)应在`__dealloc__()`方法中释放.你需要小心你在`__dealloc __()`方法中的操作.在调用`__dealloc__()`方法时，对象可能已经被部分销毁，并且对于Python而言可能不处于有效状态，如果你坚持只是释放C数据是最好的选择.

你不需要担心释放对象的Python属性，因为在`__dealloc __()`方法返回后，它将由Cython完成.
当子类化扩展类型时,请注意，超类的`__dealloc__()`方法将始终被调用，即使它被覆盖.这与典型的Python行为不同.


注意：

扩展类型没有`__del__()`方法。

## 逻辑运算方法

算术运算符方法(如`__add__()`)的行为与Python对应方法不同。这些方法没有单独的“反转”版本(`__radd __()`等).相反，如果第一个操作数不能执行操作，则调用第二个操作数的相同方法，操作数的顺序相同.


## 运算比较操作

对于不同的比较操作(`__eq__()`，`__le__()`等等)Cython没有单独的方法.而是有一个方法`__richcmp __()`，它接受一个整数，指示要执行哪种操作，如下所示:

操作|指示
---|---
<|0
==|2
>|4
<=|1
!=|3
>=|5

## 错误处理

如果你不做任何特殊的事情，用cdef声明的函数不返回任何Python对象，这样这个cdef函数就没有办法向其调用者报告其Python异常.如果在此类函数中检测到异常，则会打印一条警告消息，并忽略该异常.
如果你想要一个不返回Python对象的C函数能够将异常传播给它的调用者，你需要声明一个异常值。这里是一个例子：

```cython

cdef int spam() except -1:
    ...
```

使用此声明，每当`spam()`函数内发生异常时，它将立即返回值`-1`.此外，每当对`spam()`的调用返回`-1`时，将假定异常已经发生并将被传播.


当为函数声明异常值时，不应显式或隐式返回该值.特别是，如果异常返回值是一个`False`值，那么你应该确保函数永远不会通过隐式或空返回终止.

如果所有可能的返回值都是合法的，并且你不能完全保留一个用于信号错误，则可以使用异常值声明的替代形式：

```cython
cdef int spam() except? -1:
    ...
```

'?'号表示-1是个异常值，在这种情况下，Cython通过生成一个函数`PyErr_Occurred()`进行返回，从而知道该函数发生了异常值.


还有第三种定义方式：
```cython
cdef int spam() except *:
    ...
    
```
这种形式导致Cython在每次调用spam()后生成对`PyErr_Occurred()`的调用，而不管它返回什么值。如果你有一个函数返回void需要传播错误，你将不得不使用这种形式，因为没有任何返回值来测试.否则这种形式的定义应该尽量少用.

需要注意的是：

异常值只能为返回

+ 整数，
+ 枚举，
+ 浮点
+ 指针类型的函数声明.

并且该值必须是常量表达式。

void函数只能使用`except *`形式.

异常值规范是函数签名的一部分。如果将指针作为参数传递给函数或将其指定给变量，则声明的参数或变量的类型必须具有相同的异常值规范(或缺少).下面是一个具有异常值的指针到函数声明的示例：

```cython
int (*grail)(int, char*) except -1
```

## 模块导入

cython中的导入方式有3种:

+ python的`import`

    用于导入python模块,一般只在实现文件中导入

+ cython的`cimport`

    用于导入cpython中`.pxd`文件中申明的内容和一些cpython封装好的标准模块,可以在申明文件或者实现文件中导入

+ cython的`include`

    `include`语句用于导入`.pxi`文件.这种语法类似`C/C++`中的`#include`,是直接将目标文件内容复制到当前位置